# Setting up a model

SWAP configuration files consist of a number of switches (parameters with "SW" prefix) and related key value pairs and tables. All these can be groupped in sections corresponding to model's specific parts. For example, meteorological section contains parameters related to climatological calculations whereas Crop section handles all parameters related to, e.g., crop water uptake. This can be seen in the classic template of SWAP file imputs (see Wiki section).

In pySWAP, each distinct section is represented as a class, similarily to flopy, a popular Python package running MODFLOW models. Therefore, in a sense, defining a model with pySWAP feels similar to defining it in a classic ASCII template, but of course gives you way more options to work with your models further. Finally, when all necessary objects are defined, you create a Model instance. You can also define an empty Model instance, and add each section to that instance after it's created. Let's set up `Metadata` and `GeneralSettings` for the model. We will use the empty model instance approach.

!!! tip

    pySWAP is written with type hinting in mind. When using a code editor like Visual Studio Code, when you type `pyswap.`, you will already see hints of what class or function you can call. It works the same when you define classes; when hoover over the class signature, you will see available parameters, value ranges and documentation.

In [1]:
import pyswap as psp

In [ ]:
ml = psp.Model()

meta = psp.Metadata(
    author="John Doe",
    institution="University of Somewhere",
    email="john.doe@somewhere.com",
    project="pySWAP test - hupselbrook",
    swap_ver="4.2",
)

simset = psp.GeneralSettings(
    tstart="2002-01-01",
    tend="2004-12-31",
    extensions=["vap", "blc", "sba", "inc", "csv"],
    nprintday=1,
    swerror=1,
    swscre=0,
    swmonth=1,
    swyrvar=0,
    datefix="31 12",
    inlist_csv=[
        "rain",
        "irrig",
        "interc",
        "runoff",
        "drainage",
        "dstor",
        "epot",
        "eact",
        "tpot",
        "tact",
        "qbottom",
        "gwl",
    ],
)

ml.metadata = meta
ml.generalsettings = simset

!!! note

    At this stage there is one important difference to explain; you do not need to (and actually cannot) adjust the paths at the beginning. This is because pySWAP runs SWAP in a temporary directory and handles paths automatically.


After adding the two sections, you can view how the section would look like as a SWAP-compatible string, or see the current shape of the .swp file by calling `ml.swp` property. You may notice more parameters that you wanted to set. It's because by default, heat flow (`HeatFlow`), solute transport (`SoluteTransport`) and fixed irrigation (`FixedIrrigation`) modules are turned off in the model. There are also default settings for some parameters of Richard's equation (`RichardsSettings`). To change it, it's enough to define your own objects with desired settings and reassign them in the `ml` instance.

In [ ]:
print(ml.swp)

Setting up meteorological section additionally requires providing some rainfall data in a CSV format. Those data are enclosed in a `File` type object. For meteorological data it's `MetFile` class. Let's create the `Meteorology` object with a `MetFile` attached to it.

Currently the other built-in method of getting the MetFile is using KNMI service. Read documentation of ``

In [ ]:
# here we additionally need to load the meteo data from testcase library
from pyswap import testcase

meteo_data = psp.load_from_csv(
    metfil="283.met", csv_path=testcase.load_met("hupselbrook")
)

meteo = psp.Meteorology(
    lat=52.0,
    alt=21.0,
    swetr=0,
    metfile=meteo_data,
    swdivide=1,
    swmetdetail=0,
    altw=10.0,
    angstroma=0.25,
    angstromb=0.5,
)

ml.meteorology = meteo